In [ ]:

import tensorflow
np.random.seed(1212)
from tensorflow import keras 
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras import optimizers


In [ ]:
import cv2
import numpy as np
from skimage import feature

import matplotlib.pyplot as plt

# First take a look at the histogram of all the images in each class

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt


def hist_plot(path):
    '''
    This function provides the average of the values for each channel in the image
    
    '''
    valid_images = ["jpg","gif","png","tga","bmp"]
    results = [] 
    size = []
    y = []
    label = 0
    count=0
    for i in os.listdir(path):

            if i.split('.')[1] not in valid_images:
                continue # to loop over all images you have on the directory
            img = cv2.imread(path+i)
            gray = cv2.imread(path+i,0)
            corners = cv2.goodFeaturesToTrack(gray,4,0.2,2,useHarrisDetector=True)
            if len(corners) !=4:
                print(j)

            #img1 = img.crop((corners[1][0][0], corners[1][0][0], corners[1][0][1], corners[1][0][1])).convert('RGB')
            crners= np.unique(corners)
            crners.sort()
            size.append(img.shape[0])

            img = img[int(crners[0]):int(crners[1]), int(crners[0]):int(crners[1])]
            img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)

            count+=1
            #img = cv2.resize(img, (150,150))
            #img = np.array(img)[:, :, ::-1]
            avg_color_per_row = np.average(img, axis=0)
            avg_color = np.average(avg_color_per_row, axis=0)
            results.append(avg_color)
    np_results = np.array(results) # to make results a numpy array
    plt.xlabel('intensity')
    plt.ylabel('count')
    plt.hist(np_results,color=['blue','green','red'])
    plt.show() # to show the histogram

In [ ]:
hist_plot('train/a/')
hist_plot('train/b/')
hist_plot('train/c/')



# Resize images and create labels for the image

### Checking all valid types of the image and appending them to the arrays so that they can be used later on

In [ ]:

def create_data(path,images_fun=[],labels_fun=[],image_size=150):
    '''
    
    input is the path to the folder containing subdirectories
    
    output is the images in an array and labels ina an array
    if subdirectories are a,b,c label for images in the corresponding folders is 0,1,2
    
    '''
    IMAGE_SIZE = (image_size, image_size)
    valid_images = ["jpg","gif","png","tga","bmp"]
    label = 0
    rawImages=[]
    features=[]
    for i in os.listdir(path):
        if os.path.isdir(path + i):
            for j in os.listdir(path+i):
                if j.split('.')[1] not in valid_images:
                    continue
                #img = Image.open(path+i+'/'+j).convert('RGB')
                img = cv2.imread(path+i+'/'+j)
                gray = cv2.imread(path+i+'/'+j,0)
                corners = cv2.goodFeaturesToTrack(gray,4,0.2,2,useHarrisDetector=True)
                if len(corners) !=4:
                    print(j)

                #img1 = img.crop((corners[1][0][0], corners[1][0][0], corners[1][0][1], corners[1][0][1])).convert('RGB')
                crners= np.unique(corners)
                crners.sort()
                img = img[int(crners[0]):int(crners[1]), int(crners[0]):int(crners[1])]
                #image = cv2.imread(path+i+'/'+j)
                #image = np.array(img1)[:, :, ::-1]
                #image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

                # Append the image and its corresponding label to the output

                #pixels = image_to_feature_vector(image)
                #hist = extract_color_histogram(image)
                #image = cv2.cvtColor(img, cv2.COLOR_BGR2YCR_CB)
                image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

                image = cv2.resize(image, IMAGE_SIZE,0,0,interpolation = cv2.INTER_NEAREST) 
                images_fun.append(image)
                labels_fun.append(label)
                # update the raw images, features, and labels matricies,
                # respectively
                #rawImages.append(pixels)
                #features.append(hist)
            label+=1
    images_fun = np.array(images_fun, dtype = 'float32')
    labels_fun = np.array(labels_fun, dtype = 'int32')   
    #output = []       
    #output.append((images, labels))
    return images_fun,labels_fun,rawImages,features

images,labels,rawImages,features = create_data('train/',images_fun=[],labels_fun=[])
images_val,labels_val,raw_val,feat_val = create_data('val/',images_fun=[],labels_fun=[])

##  Display examples of dataset and view class balance

In [ ]:
def display_examples(class_names, images, labels):
    """
        Display 25 images from the images array with its corresponding labels
    """
    
    fig = plt.figure(figsize=(10,10))
    fig.suptitle("Some examples of images of the dataset", fontsize=16)
    for i in range(25):
        plt.subplot(5,5,i+1)
        plt.xticks([])
        plt.y
        ticks([])
        plt.grid(False)
        plt.imshow(images[i], cmap=plt.cm.binary)
        plt.xlabel(class_names[labels[i]])
    plt.show()

In [ ]:
import pandas as pd

_, train_counts = np.unique(labels, return_counts=True)
_, test_counts = np.unique(labels_val, return_counts=True)
pd.DataFrame({'train': train_counts,
                    'test': test_counts}, 
             index=class_names
            ).plot.bar()
plt.show()

In [ ]:
plt.pie(train_counts,
        explode=(0, 0, 0) , 
        labels=class_names,
        autopct='%1.1f%%')
plt.axis('equal')
plt.title('Proportion of each observed category')

plt.show()

In [ ]:
# normalize images 
class_names =['a','b','c']
class_names_label = {class_name:i for i, class_name in enumerate(class_names)}

nb_classes = len(class_names)


train_images = images / 255.0 
test_images = images_val / 255.0


In [ ]:
import random
combined = list(zip(train_images, labels))
random.shuffle(combined)

a, b= zip(*combined)

In [ ]:
display_examples(class_names, a, b)

# Baseline CNN

### Attempt of a simple CNN model to do the predicitons on the dataset. 

In [ ]:
# keras imports for the dataset and building our neural network
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.utils import to_categorical 

num_digits = 3
# one-hot encoding using keras' numpy-related utilities
y_train = tensorflow.keras.utils.to_categorical(labels, num_digits)
y_val = tensorflow.keras.utils.to_categorical(labels_val, num_digits)

X_train = np.array(images)
X_test = np.array(images_val)
X_train = X_train.reshape(X_train.shape[0], 150, 150, 3)
X_test = X_test.reshape(X_test.shape[0], 150, 150, 3)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# normalizing the data to help with the training
X_train /= 255.0
X_test /= 255.0

n_classes = 3
print("Shape before one-hot encoding: ", y_train.shape)
Y_train = y_train
print("Shape after one-hot encoding: ", Y_train.shape)

# building a linear stack of layers with the sequential model
model = Sequential()

# convolutional layer
model.add(Conv2D(50, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', input_shape=(150, 150, 3)))

# convolutional layer
model.add(Conv2D(100, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(100, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# flatten output of conv
model.add(Flatten())

# hidden layer
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(3, activation='softmax'))

# compiling the sequential model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

# training the model for 15 epochs
model.fit(X_train, Y_train, batch_size=16, epochs=15,validation_data=(X_test,y_val))

# 

# Extract features with Vgg-16 and perform PCA

Using the VGG19, extract the features of the dataset. Then use PCA on the featurees to see if clusters of the dataset is being formed by the model

In [ ]:
from tensorflow.keras.applications.resnet import ResNet101
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16
import tensorflow as tf
from tensorflow.keras.applications.vgg19 import VGG19

#model = resnet50.ResNet50(weights='imagenet', include_top=False)
model2 = VGG19(weights='imagenet',include_top= False,input_shape = (150,150,3))


In [ ]:
train_features = model2.predict(train_images)
test_features = model2.predict(test_images)

In [ ]:
test_features.shape

In [ ]:
n_train, x, y, z = train_features.shape
n_test, x, y, z = test_features.shape
numFeatures = x * y * z

In [ ]:
from sklearn import decomposition

pca = decomposition.PCA(n_components = 7)

X_test = test_features.reshape((n_train, x*y*z))

pca.fit(X)

C = pca.transform(X)
C_test = pca.transform(X_test)

C1 = C[:,0]
C2 = C[:,1]

In [ ]:

plt.subplots(figsize=(10,10))

for i, class_name in enumerate(class_names):
    plt.scatter(C1[labels == i][:1000], C2[labels == i][:1000], label = class_name, alpha=0.4)
plt.legend()
plt.title("PCA Projection")
plt.show()

In [ ]:
import tensorflow as tf
model2 = tf.keras.Sequential([
    
    tf.keras.layers.Flatten(input_shape = (x, y, z)),
    tf.keras.layers.Dense(50, activation=tf.nn.leaky_relu),
    tf.keras.layers.Dense(50, activation=tf.nn.leaky_relu),

    tf.keras.layers.Dense(3, activation=tf.nn.softmax)
])

model2.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

history2 = model2.fit(train_features, labels, batch_size=64, epochs=50)

In [ ]:
label_pred = model2.predict(test_features)
label_pred  = np.argmax(label_pred, axis = 1)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(labels_val,label_pred))
print(classification_report(labels_val,label_pred))
print(accuracy_score(labels_val, label_pred))

In [ ]:
def plot_accuracy_loss(history):
    """
        Plot the accuracy and the loss during the training of the nn.
    """
    fig = plt.figure(figsize=(10,5))

    # Plot accuracy
    plt.subplot(221)
    plt.plot(history.history['accuracy'],'bo--', label = "acc")
    plt.plot(history.history['val_accuracy'], 'ro--', label = "val_acc")
    plt.title("train_acc vs val_acc")
    plt.ylabel("accuracy")
    plt.xlabel("epochs")
    plt.legend()

    # Plot loss function
    plt.subplot(222)
    plt.plot(history.history['loss'],'bo--', label = "loss")
    plt.plot(history.history['val_loss'], 'ro--', label = "val_loss")
    plt.title("train_loss vs val_loss")
    plt.ylabel("loss")
    plt.xlabel("epochs")

    plt.legend()
    plt.show()

In [ ]:
plot_accuracy_loss(history2)


## Connect SVM to the PCA components

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.svm import SVC

svm_clf = SVC(C=5,kernel='rbf',tol=1e-7,gamma=0.3)
svm_clf.fit(C,labels)
import pandas as pd
svm = svm_clf.predict(C_test)
df_confusion = pd.crosstab(labels_val, svm,rownames=['actual'],colnames=['predicted'])
df_confusion

# Using quantitative features 

In [ ]:
df_tr = pd.DataFrame(columns=['size_im_x','size_sq_x','r','g','b','y','cr','cb','un','label'])

In [ ]:

df_val = pd.DataFrame(columns=['size_im_x','size_sq_x','r','g','b','y','cr','cb','un,''label'])

In [ ]:
count=0
def create_data_pandas(path,df,image_size=150):
    
    '''
    Create a data frame with quantitative features
    
    '''
    valid_images = ["jpg","gif","png","tga","bmp"]
    label = 0

    for i in os.listdir(path):
        if os.path.isdir(path + i):
            for j in os.listdir(path+i):
                if j.split('.')[1] not in valid_images:
                    continue
                #img = Image.open(path+i+'/'+j).convert('RGB')
                img = cv2.imread(path+i+'/'+j)
                gray = cv2.imread(path+i+'/'+j,0)
                corners = cv2.goodFeaturesToTrack(gray,4,0.2,2,useHarrisDetector=True)
                if len(corners) !=4:
                    print(j)
                im_x,im_y = img.shape[0],img.shape[1]
                #img1 = img.crop((corners[1][0][0], corners[1][0][0], corners[1][0][1], corners[1][0][1])).convert('RGB')
                crners= np.unique(corners)
                crners.sort()
                un = np.unique(img)
                img = img[int(crners[0]):int(crners[1]), int(crners[0]):int(crners[1])]
                sq_x,sq_y = img.shape[0],img.shape[1]
                avg_color_per_row = np.average(img, axis=0)
                b,g,r = np.average(avg_color_per_row, axis=0)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2YCR_CB)
                avg_color_per_row = np.average(img, axis=0)
                y,cr,cb = np.average(avg_color_per_row, axis=0)
                
                df = df.append({'size_im_x':im_x,'size_sq_x':sq_x,'r':r,'g':g,'b':b,'y':y,'cr':cr,'cb':cb,'un':un,'label':label}, ignore_index=True)
                #image = cv2.imread(path+i+'/'+j)
                #image = np.array(img1)[:, :, ::-1]
                #image = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)

                # update the raw images, features, and labels matricies,
                # respectively

            label+=1

    #output = []       
    #output.append((images, labels))
    return df

df_tr = create_data_pandas('train/',df_tr)
df_val = create_data_pandas('val/',df_val)

In [ ]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
df_tr = df_tr.sample(frac=1)

sc = StandardScaler()
X_tr = sc.fit_transform(df_tr.iloc[:,:-1].values)
X_te = sc.transform(df_val.iloc[:,:-1].values)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

regressor = RandomForestClassifier(n_estimators=100, random_state=0)
regressor.fit(df_tr.iloc[:,:-1].values, df_tr['label'])
y_pred = regressor.predict(X_te)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(df_val['label'],y_pred))
print(classification_report(df_val['label'],y_pred))
print(accuracy_score(df_val['label'], y_pred))

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.svm import SVC

svm_clf = SVC(C=5,kernel='rbf',gamma=0.2,tol=1e-7)
svm_clf.fit(X_tr,df_tr['label'])
y_pred = svm_clf.predict(X_te)

## Error Analysis

## We check the misclassified images and analyze them to find any pattern

In [ ]:
def print_mislabeled_images(class_names, test_images, test_labels, pred_labels):
    """
        Print 25 examples of mislabeled images by the classifier, e.g when test_labels != pred_labels
    """
    BOO = (test_labels == pred_labels)
    mislabeled_indices = np.where(BOO == 0)
    mislabeled_images = test_images[mislabeled_indices]
    mislabeled_labels = pred_labels[mislabeled_indices]

    title = "Some examples of mislabeled images by the classifier:"
    display_examples(class_names,  mislabeled_images, mislabeled_labels)
print_mislabeled_images(class_names, test_images, labels_val, label_pred)

In [ ]:
f = []
for j in os.listdir('val/b'):
    f.append(j)

In [ ]:
import glob
for i in np.where((labels_val!=label_pred) & (labels_val==0) & (label_pred==1))[0]:
    img_name = f[i]
    print(img_name)
    plt.imshow(cv2.imread('val/a/' + img_name)[:, :, ::-1])
    plt.show()

In [ ]:
import glob
for i in np.where((labels_val!=label_pred) & (labels_val==1) & (label_pred==0))[0]:
    img_name = f[i-500]
    print(img_name)
    plt.imshow(cv2.imread('val/b/' + img_name)[:, :, ::-1])
    plt.show()

In [ ]:
results2 = []
size = []
for i in np.where((labels_val!=label_pred) & (labels_val==1) & (label_pred==0))[0]:
    
    avg_color_per_row2 = np.average(images_val[i], axis=0)
    size.append(images_val[i].shape[0])
    avg_color2 = np.average(avg_color_per_row2, axis=0)
    results2.append(avg_color2)
np_results2 = np.array(results2) # to make results a numpy array
plt.hist(np_results2,color=['red','green','blue'])
plt.show() # to show the histogram

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(df_val['label'],y_pred))
print(classification_report(df_val['label'],y_pred))
print(accuracy_score(df_val['label'], y_pred))

In [ ]:
import glob
im = []
for i in np.where((df_val['label']!=y_pred) & (df_val['label']==1) & (y_pred==0))[0]:
    img_name = f[i-500]
    print(img_name)
    plt.imshow(cv2.imread('val/b/' + img_name)[:, :, ::-1])
    im.append(cv2.imread('val/b/' + img_name))
    plt.show()

## Error Analysis for quantitative


In [ ]:
import glob
im2=[]
for i in np.where((df_val['label']!=y_pred) & (df_val['label']==0) & (y_pred==1))[0]:
    img_name = f[i]
    print(img_name)
    plt.imshow(cv2.imread('val/a/' + img_name)[:, :, ::-1])
    im2.append(cv2.imread('val/a/' + img_name))
    plt.show()

In [ ]:
results2 = []
size = []
for i in np.where((labels_val!=label_pred) & (labels_val==1) & (label_pred==0))[0]:
    
    avg_color_per_row2 = np.average(images_val[i], axis=0)
    size.append(images_val[i].shape[0])
    avg_color2 = np.average(avg_color_per_row2, axis=0)
    results2.append(avg_color2)
np_results2 = np.array(results2) # to make results a numpy array
plt.hist(np_results2,color=['red','green','blue'])
plt.show() # to show the histogram